## Introduction

This notebook goes through the process using tweepy to collect streaming tweets, then uploads those tweets to an existing postgres database in Linux.

## Tweepy and the twitter streaming api

Twitter has an api that lets you access stored tweets in a limited way or an active stream of live tweets that can be filtered. Tweepy is the most popular python library for interacting with this api. It handles the authorization for you, and has a streamlistener class that you need to customize to use. 

## Authorization

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 28 13:13:38 2017

@author: EPY
"""
import time
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy import StreamListener
import os
import json


In [2]:
cons_key = 
cons_secret = 
access_token = 
access_token_secret = 
auth = OAuthHandler(cons_key, cons_secret)
auth.set_access_token(access_token, access_token_secret)

## The streamelistener class

In [5]:
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        #The try/except statements were used here to filter out retweets 
        try:
            status.retweeted_status
        except:     
            if status.in_reply_to_user_id is None:           
                #print(status.text)
                tweet = status._json
                self.file.write( json.dumps(tweet) + '\n' )
                self.num_tweets += 1
                if self.num_tweets <= 1000:
                    return True
                else:
                    return False
                self.file.close()

    def on_error(self, status):
        print(status)

In [6]:
#wonder woman grabs tweets that have 'wonderwoman' and 'wonder woman'
keyword_list = ['wonder woman']
twitterStream = Stream(auth, MyStreamListener()) #initialize Stream object with a time out limit

twitterStream.filter(track=keyword_list, languages=['en'])  #call the filter method to run the Stream Object

## Parsing json strings

The stream of tweets has been saved as tweets.txt, each tweet is saved as a json file with a number of attributes. In order to read the file I go through the text file tweet by tweet and convert each json string to a dictionary and put them all in a list. Finally, the list is converted to a pandas dataframe.

In [7]:
import numpy as np

In [1]:
import json
#each line is a complete json object
with open('tweets.txt', 'r') as f:
    tweet_list = []
    for line in f:
        tweets = json.loads(line)
        tweet_list.append(tweets)
    
print(tweet_list[0].keys())    


dict_keys(['retweet_count', 'quoted_status_id', 'is_quote_status', 'in_reply_to_screen_name', 'truncated', 'favorited', 'contributors', 'retweeted', 'id_str', 'text', 'geo', 'entities', 'quoted_status', 'place', 'lang', 'timestamp_ms', 'id', 'in_reply_to_user_id', 'source', 'coordinates', 'possibly_sensitive', 'created_at', 'in_reply_to_status_id_str', 'filter_level', 'favorite_count', 'user', 'in_reply_to_user_id_str', 'quoted_status_id_str', 'in_reply_to_status_id'])


In [2]:
import pandas as pd
t = pd.DataFrame(tweet_list)

In [3]:
#print(t.head())
t.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted', 'source', 'text',
       'timestamp_ms', 'truncated', 'user'],
      dtype='object')

In [9]:
t[['in_reply_to_status_id', 'created_at', 'in_reply_to_user_id_str', 'text']].head(10)

,in_reply_to_status_id,created_at,in_reply_to_user_id_str,text
0,None,Tue Jun 06 14:20:14 +0000 2017,None,JERUSALEMPOST reports Jordan considering banni...
1,None,Tue Jun 06 14:20:21 +0000 2017,None,How WONDER WOMAN Inspires Leaving Your Safe Sp...
2,None,Tue Jun 06 14:20:24 +0000 2017,None,I may be marvel af but Wonder Woman is my favo...
3,None,Tue Jun 06 14:20:27 +0000 2017,None,The Complex Gender Politics of the 'Wonder Wom...
4,None,Tue Jun 06 14:20:29 +0000 2017,None,"Oh. Right. Yes, that definitely has to happen ..."
5,None,Tue Jun 06 14:20:29 +0000 2017,None,https://t.co/oaRFItq0N2 @BuncheUCLA 2017 #Holl...
6,None,Tue Jun 06 14:20:31 +0000 2017,None,The turnout to my BBQ was one of the best ever...
7,None,Tue Jun 06 14:20:32 +0000 2017,None,"The more I think about Wonder Woman, the more ..."
8,None,Tue Jun 06 14:20:34 +0000 2017,None,Lets discuss! https://t.co/XygbnJo6z8
9,None,Tue Jun 06 14:20:34 +0000 2017,None,And can I just say I've never wanted to be an ...


Some columns are actually multiple values, in the case of the entities key they are dicts. You'll need to convert these if you want to store them in a SQL database like postgres. For my purposes, I really only need the texts though.

In [12]:
t['entities'][0]

{'hashtags': [],
 'symbols': [],
 'urls': [{'display_url': 'twitter.com/Jerusalem_Post…',
   'expanded_url': 'https://twitter.com/Jerusalem_Post/status/872094509194035201',
   'indices': [94, 117],
   'url': 'https://t.co/kjYL4EfQID'}],
 'user_mentions': []}

In [5]:
df = t[['created_at', 'favorite_count', 'text']]
df.head(20)

,created_at,favorite_count,text
0,Tue Jun 06 14:20:14 +0000 2017,0,JERUSALEMPOST reports Jordan considering banni...
1,Tue Jun 06 14:20:21 +0000 2017,0,How WONDER WOMAN Inspires Leaving Your Safe Sp...
2,Tue Jun 06 14:20:24 +0000 2017,0,I may be marvel af but Wonder Woman is my favo...
3,Tue Jun 06 14:20:27 +0000 2017,0,The Complex Gender Politics of the 'Wonder Wom...
4,Tue Jun 06 14:20:29 +0000 2017,0,"Oh. Right. Yes, that definitely has to happen ..."
5,Tue Jun 06 14:20:29 +0000 2017,0,https://t.co/oaRFItq0N2 @BuncheUCLA 2017 #Holl...
6,Tue Jun 06 14:20:31 +0000 2017,0,The turnout to my BBQ was one of the best ever...
7,Tue Jun 06 14:20:32 +0000 2017,0,"The more I think about Wonder Woman, the more ..."
8,Tue Jun 06 14:20:34 +0000 2017,0,Lets discuss! https://t.co/XygbnJo6z8
9,Tue Jun 06 14:20:34 +0000 2017,0,And can I just say I've never wanted to be an ...


## Connecting to postgres with psycopg2

Psycopg2 is a driver designed only for postgres and allows you to pass on SQL code to interact with the database, whereas sqlalchemy provides an orm model and a way to connect to postgres. 

In [6]:
import psycopg2

try:
    connect_str = "dbname='' user='' host='' " + \
                  "password=''"
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    # create a psycopg2 cursor that can execute queries
    cursor = conn.cursor()
    # create a new table with a single column called "name"
    cursor.execute("""CREATE TABLE ww (name char(40));""")
    # run a SELECT statement - no data in there, but we can try it
    cursor.execute("""SELECT * from ww""")
    rows = cursor.fetchall()
    print(rows)
except Exception as e:
    #print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

Uh oh, can't connect. Invalid dbname, user or password?
relation "ww" already exists



In [7]:
conn.close()

## Connecting to postgres with sqlalchemy

In [1]:

user='' 
host=':' 
password=''
dbname = ''

url = 'postgresql://{}:{}@{}/{}'
url = url.format(user, password, host, dbname)

In [3]:
import pandas as pd
from sqlalchemy import create_engine, MetaData

engine = create_engine(url)
conn = engine.connect()

#df.to_sql("wonder", con = engine, if_exists = 'replace')
meta = MetaData()
meta.create_all(engine)
result = conn.execute("SELECT * FROM wonder LIMIT 100")

In [5]:
ww_tweets = []
for row in result:
    ww_tweets.append(row)
ww_tweets    

[(0, 'Tue Jun 06 14:20:14 +0000 2017', 0, 'JERUSALEMPOST reports Jordan considering banning ‘Wonder Woman’ over Israeli star Gal Gadot … https://t.co/kjYL4EfQID'),
 (1, 'Tue Jun 06 14:20:21 +0000 2017', 0, 'How WONDER WOMAN Inspires Leaving Your Safe Space | Nerdist #ParadiseIsland #Iretreat https://t.co/BN966Dzaz7'),
 (2, 'Tue Jun 06 14:20:24 +0000 2017', 0, 'I may be marvel af but Wonder Woman is my favorite superhero film'),
 (3, 'Tue Jun 06 14:20:27 +0000 2017', 0, "The Complex Gender Politics of the 'Wonder Woman' Movie https://t.co/KDJycoQw3d via @thr"),
 (4, 'Tue Jun 06 14:20:29 +0000 2017', 0, 'Oh. Right. Yes, that definitely has to happen first. https://t.co/mFbBqcV6QV'),
 (5, 'Tue Jun 06 14:20:29 +0000 2017', 0, 'https://t.co/oaRFItq0N2 @BuncheUCLA 2017 #HollywoodDiversity Report cited in story on  #WonderWoman box office success.'),
 (6, 'Tue Jun 06 14:20:31 +0000 2017', 0, 'The turnout to my BBQ was one of the best ever on a non-olympic-year Tuesday occurring in the first w

In [27]:
type(result)

sqlalchemy.engine.result.ResultProxy

In [5]:
engine.table_names()

['wonder', 'ryzen']

## Viewing column names

In [7]:
from sqlalchemy import Table, inspect

In [8]:
#ryzen = Table('ryzen', meta, autoload = True, autoload_with = True)
#a = [c.name for c in ryzen.columns]
inspector=inspect(engine)
inspector.get_columns('wonder')

[{'autoincrement': False,
  'default': None,
  'name': 'index',
  'nullable': True,
  'type': BIGINT()},
 {'autoincrement': False,
  'default': None,
  'name': 'created_at',
  'nullable': True,
  'type': TEXT()},
 {'autoincrement': False,
  'default': None,
  'name': 'favorite_count',
  'nullable': True,
  'type': BIGINT()},
 {'autoincrement': False,
  'default': None,
  'name': 'text',
  'nullable': True,
  'type': TEXT()}]

In [9]:
for dict in inspector.get_columns('wonder'):
    print(dict['name'])

index
created_at
favorite_count
text


In [10]:
result = conn.execute("SELECT * FROM wonder")

## Delete a table

In [13]:
conn.execute("DROP TABLE wonder")

## Add a column

In [41]:
conn.execute("ALTER TABLE wonder \
             ADD COLUMN new_column character varying(50) \
             NOT NULL DEFAULT 'foo'")


## Rename a column

In [49]:
conn.execute("ALTER TABLE wonder \
             RENAME new_column TO renamed_column")


In [ ]:
conn.close()